In [ ]:
import matplotlib.pyplot as plt
import requests_cache
import moonbox

In [ ]:
session = requests_cache.CachedSession("cache")
year = 2025

calendar = moonbox.calendar(year=year, session=session)

In [ ]:
plt.ioff()

# set up the calendar geometry
canvas_size = (28.0, 22.0)
calendar_width_prop = 0.95
spacer_x_prop = 0.2
spacer_y_prop = 1.5
calendar_middle = 0.6  # middle of calendar is this fraction down the canvas

date_font_size = 20
title_font_size = 60
signature_font_size = 30

# data
n_months = calendar[-1]["lunar_month"] + 1
n_days = 30  # maximum length of a lunar month

# derived positions
calendar_width = canvas_size[0] * calendar_width_prop
moon_radius = calendar_width / (2 * n_days + spacer_x_prop * (n_days - 1))
spacer_x = spacer_x_prop * moon_radius
spacer_y = spacer_y_prop * moon_radius

calendar_height = n_months * 2 * moon_radius + (n_months - 1) * spacer_y

calendar_x = (canvas_size[0] - calendar_width) / 2
calendar_y = canvas_size[1] - (canvas_size[1] - calendar_height) * calendar_middle

fig = plt.figure(figsize=canvas_size)
ax = fig.add_axes([0, 0, 1, 1])
ax.set_xlim(0, canvas_size[0])
ax.set_ylim(0, canvas_size[1])

ax.set_facecolor("black")

solar_month = None

for datum in calendar:
    if (
        (datum["date"].month == 1 and datum["date"].day == 1)
        or datum["lunar_day"] == 0
        or datum["date"].day == 1
    ):
        label = datum["date"].strftime("%b %-d")
        date_align = "left"
    else:
        label = datum["date"].strftime("%-d")
        date_align = "center"

    f = datum["illumination"] / 100
    direction = datum["phase"].split()[0].lower()

    moon_x = (
        calendar_x + (2 * moon_radius + spacer_x) * datum["lunar_day"] + moon_radius
    )
    # note that y decreases as months increase
    moon_y = (
        calendar_y - (2 * moon_radius + spacer_y) * datum["lunar_month"] - moon_radius
    )

    moonbox.draw_moon(
        ax,
        x=moon_x,
        y=moon_y,
        radius=moon_radius,
        f=f,
        direction=direction,
        dark="0.2",
    )
    # date labels
    if date_align == "left":
        date_x = moon_x - moon_radius
    elif date_align == "center":
        date_x = moon_x
    else:
        raise RuntimeError

    ax.text(
        x=date_x,
        y=moon_y + moon_radius,
        s=label,
        horizontalalignment=date_align,
        verticalalignment="bottom",
        color="white",
        fontsize=date_font_size,
    )

# title
ax.text(
    x=15.0,
    y=20.0,
    s=f"{year} Lunar Months",
    verticalalignment="top",
    horizontalalignment="center",
    fontsize=title_font_size,
    color="white",
)

# signature
ax.text(
    x=canvas_size[0],
    y=0.0,
    s="Scott Olesen",
    verticalalignment="bottom",
    horizontalalignment="right",
    fontsize=signature_font_size,
    color="white",
)

# do not show the figure; rendering in VSCode is very slow
fig.savefig("../output/tmp.png", dpi=72)

In [ ]:
# production quality
fig.savefig("../output/lunar_calendar.png", dpi=300)
fig.savefig("../output/lunar_calendar.pdf")